In [ ]:
import altair as alt
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
tickers_of_interest = ["AAPL", "AMZN", "GOOGL", "MSFT", "TSLA"]

In [ ]:
tickers = yf.Tickers(tickers_of_interest)

In [ ]:
hist = tickers.history(period="5y")

In [ ]:
hist.tail()

In [ ]:
hist_df = pd.DataFrame(hist.loc[:, ("Close", "AAPL")])
hist_df.columns = ["Close"]

hist_df = hist_df.reset_index()

In [ ]:
short_sma = 1
short_sma_column = f"SMA_{short_sma}"
long_sma = 200
long_sma_column = f"SMA_{long_sma}"
hist_df[short_sma_column] = hist_df["Close"].rolling(short_sma).mean()
hist_df[long_sma_column] = hist_df["Close"].rolling(long_sma).mean()
hist_df["Position"] = np.where(hist_df[short_sma_column] > hist_df[long_sma_column], 1, -1)
hist_df.dropna(inplace=True)

hist_df["Returns"] = np.log(hist_df["Close"]/hist_df["Close"].shift(1))
hist_df["Strategy"] = hist_df["Position"].shift(1) * hist_df["Returns"]
hist_df["Returns_Sum"] = hist_df["Returns"].cumsum().apply(np.exp)
hist_df["Strategy_Sum"] = hist_df["Strategy"].cumsum().apply(np.exp)

In [ ]:
hist_df.loc[:, ["Returns", "Strategy"]].sum().apply(np.exp)

In [ ]:
hist_df

In [ ]:
plot_df = hist_df.loc[:, ["Date", "Close", short_sma_column, long_sma_column]].melt("Date", var_name="Type", value_name="Price")

base = alt.Chart(plot_df).mark_line().encode(
    x="Date:T",
    y="Price:Q",
    color="Type:N"
)

base

In [ ]:
plot_df = hist_df.loc[:, ["Date", "Position", "Returns_Sum", "Strategy_Sum"]].melt("Date", var_name="Type", value_name="Returns")

base = alt.Chart(plot_df).mark_line().encode(
    x="Date:T",
    y="Returns:Q",
    color="Type:N"
)

base

## S&P 500 components 

In [ ]:
sp500_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

In [ ]:
assert len(sp500_df) == 505

In [ ]:
sp500_tickers = yf.Tickers(list(sp500_df["Symbol"]))

In [ ]:
sp500_hist = sp500_tickers.history(period="1y")